In [5]:
import sys
import os
import numpy as np
from importlib import reload

# Adjust path to point to src directory
current_dir = os.getcwd()

# Assuming src is two levels up from the current directory
src_path = os.path.abspath(os.path.join(current_dir, '../../backend/match'))
sys.path.append(src_path)
src_path = os.path.abspath(os.path.join(current_dir, '../../backend/'))
sys.path.append(src_path)

In [8]:
src_path

'/home/david/Documents/data_science/projects/soccer-stars-django-webapp/server/backend'

In [9]:
from match_logic import utils, game_entities, collision_resolver, motion

# Reloads
utils = reload(utils)
game_entities = reload(game_entities)
collision_resolver = reload(collision_resolver)
motion = reload(motion)

# Import classes
ur = utils.UtilsRender()
CollisionResolver = collision_resolver.CollisionResolver
Motion = motion.Motion

# Run motion with 10 caps with friction 

In [10]:
def setup_aligned_caps(num_caps, boundaries=(1920,1080), radii=25):
    # Calculate the vertical step to distribute caps evenly in the height of 1080
    w,h = boundaries
    step_y = (h - 2 * radii) / (num_caps - 1)
    
    # Initialize positions, velocities, masses, and radii
    X = np.array([[radii + 15, radii + i * step_y] for i in range(num_caps)], dtype=np.float32)
    R = np.array([radii] * num_caps, dtype=np.float32)
    M = np.array([1] * num_caps, dtype=np.float32)
    V = np.array([[i + 1, 0] for i in range(num_caps)], dtype=np.float32)

    # Round coordinates
    X = np.round(X, 2)
    V = np.round(V, 2)
    
    return X, R, M, V

In [11]:
# Manual setup
num_caps = 8
w = 1920
h = 1080
boundaries = (w, h)
ratio_radii_cap_h = 1./17
radii = int(h * ratio_radii_cap_h)
print("Radii:", radii)
X = np.array([
     [radii + 5, 150],
     [radii + 5, 250],
     [radii + 5, 350],
     [radii + 5, 450],
     [radii + 5, 550],
     [radii + 5, 650],
     [radii + 5, 750],
     [radii + 5, 850]], dtype=np.float32
)

R = np.array([radii] * 10, dtype=np.float32)
M = np.array([1] * 10, dtype=np.float32)

V = np.array(
    [[1, 0],
     [2, 0],
     [3, 0],
     [4, 0],
     [5, 0],
     [6, 0],
     [7, 0],
     [8, 0],
     [9, 0],
     [10, 0]], dtype=np.float32
)


X, R, M, V = setup_aligned_caps(num_caps=num_caps, boundaries=boundaries, radii=radii)

Radii: 63


In [9]:
ur.render_snapshot(X, R)

In [10]:
motion = Motion(R=R, M=M, boundaries=boundaries)

In [11]:
Xs, Vs, latency = motion.simulate_motion(X=X, V=V)
print(f"Latency: {latency} ms")

Latency: 28 ms


In [12]:
ur.render_motion(positions=Xs, radius=R, add_delay=5)

In [13]:
Xmat = np.stack(Xs)

In [14]:
t = 250
ur.render_snapshot(position=Xmat[t, ...], radius=R)

# Run snapshot field with goals depicted

We want to verify that the new way to encode the goal and its collisions work

In [15]:
# Manual setup
num_caps = 1
w = 1920
h = 1080
boundaries = (w, h)
ratio_radii_cap_h = 1./17
radii = int(h * ratio_radii_cap_h)
print("Radii:", radii)
X = np.array([
     [radii + 5, 550],
     ], dtype=np.float32
)

R = np.array([radii] * num_caps, dtype=np.float32)
M = np.array([1] * num_caps, dtype=np.float32)

V = np.array([
    [-10, 0]], dtype=np.float32
)

Radii: 63


In [16]:
# ur.render_field_snapshot(X, R)

In [17]:
motion = Motion(R=R, M=M, boundaries=boundaries)
Xs, Vs, latency = motion.simulate_motion(X=X, V=V)


In [18]:
ur.render_motion(positions=Xs, radius=R, add_delay=5)

# Debug Field Collisions

In [19]:
from match_logic import utils, game_entities, collision_resolver, motion

# Reloads
utils = reload(utils)
game_entities = reload(game_entities)
collision_resolver = reload(collision_resolver)
motion = reload(motion)

# Import classes
ur = utils.UtilsRender()
CollisionResolver = collision_resolver.CollisionResolver
Motion = motion.Motion

In [20]:
# Manual setup
n = 1
w = 1920
h = 1080
margin = 300
boundaries = (w, h)
ratio_radii_cap_h = 1./17
radii = int(h * ratio_radii_cap_h)
goal_size = int(radii * 6)
goal_depth = int(radii * 2)
print("Radii:", radii)
X = np.array([
     [-30, 550],
     ], dtype=np.float32
)

R = np.array([radii] * num_caps, dtype=np.float32)
M = np.array([1] * num_caps, dtype=np.float32)

V = np.array([
    [-10, 0]], dtype=np.float32
)

Radii: 63


In [21]:
cr = CollisionResolver(R=R, M=M, boundaries=boundaries, goal_depth=goal_depth, goal_size=goal_size)
ur = utils.UtilsRender(window_size=boundaries, goal_depth=goal_depth, goal_size=goal_size)

### Left Net Collision

In [22]:
X = np.array([
     [-60, 550],
     ], dtype=np.float32
)
V = np.array([
    [-10, 0]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [23]:
ur.render_field_snapshot(X, R, margin=margin)

In [24]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [25]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [26]:
colliding_indices

[(0, 0)]

In [27]:
colliding_edges

[(0, 0)]

### Left Net Top Lateral collision

In [28]:
X = np.array([
     [-50, 420],
     ], dtype=np.float32
)
V = np.array([
    [0, -20]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [183]:
ur.render_field_snapshot(X, R, margin=margin)

In [168]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [184]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [185]:
colliding_indices

[(0, 1)]

In [186]:
colliding_edges

[(0, 1)]

### Left Net Bottom Lateral collision

In [193]:
X = np.array([
     [-50, 660],
     ], dtype=np.float32
)
V = np.array([
    [0, 20]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [194]:
ur.render_field_snapshot(X, R, margin=margin)

In [195]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [196]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [197]:
colliding_indices

[(0, 2)]

In [198]:
colliding_edges

[(0, 3)]

### [Multiple] Left Net + Left Net Bottom Lateral collision

In [218]:
X = np.array([
     [-60, 660],
     ], dtype=np.float32
)
V = np.array([
    [-10, 20]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [219]:
ur.render_field_snapshot(X, R, margin=margin)

In [203]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [220]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [221]:
colliding_indices

[(0, 0), (0, 2)]

In [222]:
colliding_edges

[(0, 0), (0, 3)]

### Top Net Left Lateral

In [235]:
X = np.array([
     [70, 250],
     ], dtype=np.float32
)
V = np.array([
    [-10, 0]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [236]:
ur.render_field_snapshot(X, R, margin=margin)

In [228]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [237]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [238]:
colliding_indices

[(0, 3)]

In [239]:
colliding_edges

[(0, 0)]

### Top Net Right Lateral

In [242]:
X = np.array([
     [70, 850],
     ], dtype=np.float32
)
V = np.array([
    [-10, 0]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [243]:
ur.render_field_snapshot(X, R, margin=margin)

In [244]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [245]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [246]:
colliding_indices

[(0, 4)]

In [247]:
colliding_edges

[(0, 0)]

### Top

In [252]:
X = np.array([
     [500, 67],
     ], dtype=np.float32
)
V = np.array([
    [10, -10]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [253]:
ur.render_field_snapshot(X, R, margin=margin)

In [ ]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [254]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [255]:
colliding_indices

[(0, 10)]

In [256]:
colliding_edges

[(0, 1)]

### Bottom

In [261]:
X = np.array([
     [500, 1010],
     ], dtype=np.float32
)
V = np.array([
    [10, 10]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [262]:
ur.render_field_snapshot(X, R, margin=margin)

In [263]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [264]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [265]:
colliding_indices

[(0, 11)]

In [266]:
colliding_edges

[(0, 3)]

# Debug Field Motion Collisions

In [29]:
from match_logic import utils, game_entities, collision_resolver, motion

# Reloads
utils = reload(utils)
game_entities = reload(game_entities)
collision_resolver = reload(collision_resolver)
motion = reload(motion)

# Import classes
ur = utils.UtilsRender()
CollisionResolver = collision_resolver.CollisionResolver
Motion = motion.Motion

In [30]:
# Manual setup
n = 1
w = 1920
h = 1080
margin = 300
boundaries = (w, h)
ratio_radii_cap_h = 1./17
radii = int(h * ratio_radii_cap_h)
goal_size = int(radii * 6)
goal_depth = int(radii * 2)
print("Radii:", radii)
X = np.array([
     [-30, 550],
     ], dtype=np.float32
)

R = np.array([radii] * n, dtype=np.float32)
M = np.array([1] * n, dtype=np.float32)

V = np.array([
    [-10, 0]], dtype=np.float32
)

Radii: 63


In [31]:
ur = utils.UtilsRender(window_size=boundaries, goal_depth=goal_depth, goal_size=goal_size)

### Simulate Field Motion

In [32]:
motion = Motion(
    R=R,
    M=M,
    min_velocity=0.1,
    boundaries=boundaries,
    goal_size=goal_size,
    goal_depth=goal_depth
)

In [33]:
%%time
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

CPU times: user 35.1 ms, sys: 12.9 ms, total: 48 ms
Wall time: 41.2 ms


In [34]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=5)

### Left Net Collision

In [35]:
X = np.array([
     [700, 550],
     ], dtype=np.float32
)
V = np.array([
    [-10, 0]], dtype=np.float32
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

In [36]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=5)

### Left Net Top and Bottom Lateral collision

In [37]:
X = np.array([
     [-50, 420],
     ], dtype=np.float32
)
V = np.array([
    [0, -5]], dtype=np.float32
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

In [38]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=10)

invalid command name "125979182366656update_position"
    while executing
"125979182366656update_position"
    ("after" script)


### [Multiple] Left Net + Left Net Bottom Lateral collision

In [41]:
X = np.array([
     [-60, 660],
     ], dtype=np.float32
)
V = np.array([
    [-1, 2]], dtype=np.float32
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

In [42]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=10)

### Top Net Left Lateral

In [43]:
X = np.array([
     [70, 250],
     ], dtype=np.float32
)
V = np.array([
    [-1, 0]], dtype=np.float32
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

In [44]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=10)

invalid command name "125979182390400update_position"
    while executing
"125979182390400update_position"
    ("after" script)


### Top Net Right Lateral

In [45]:
X = np.array([
     [70, 850],
     ], dtype=np.float32
)
V = np.array([
    [-10, 0]], dtype=np.float32
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

In [46]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=10)

### Top

In [49]:
X = np.array([
     [500, 67],
     ], dtype=np.float32
)
V = np.array([
    [10, -10]], dtype=np.float32
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

In [50]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=10)

invalid command name "125979182378688update_position"
    while executing
"125979182378688update_position"
    ("after" script)


### Bottom

In [ ]:
X = np.array([
     [500, 1010],
     ], dtype=np.float32
)
V = np.array([
    [10, 10]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [ ]:
ur.render_field_snapshot(X, R, margin=margin)

In [ ]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [ ]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [ ]:
colliding_indices

[(0, 11)]

In [ ]:
colliding_edges

[(0, 3)]

# Debug Match Collisions

In [65]:
from match_logic import utils, game_entities, collision_resolver, motion, formations

# Reloads
utils = reload(utils)
game_entities = reload(game_entities)
collision_resolver = reload(collision_resolver)
motion = reload(motion)
formations = reload(formations)

# Import classes
ur = utils.UtilsRender()
CollisionResolver = collision_resolver.CollisionResolver
Motion = motion.Motion
FormationProducer = formations.FormationProducer


In [66]:
# Match params
n = 1
w = 1920
h = 1080
margin = 300
boundaries = (w, h)

# This is the standardized radii size (for the 29 cm x 17 cm field)
cap_radii = 1.
cap_mass = 1
ratio_radii_cap_h = cap_radii/17

# This is the radii for current field in pixels (not in cm)
radii = int(h * ratio_radii_cap_h)
goal_size = int(radii * 6)
goal_depth = int(radii * 2)
ur = utils.UtilsRender(window_size=boundaries, goal_depth=goal_depth, goal_size=goal_size)

In [67]:
# Match formation
formation_producer = FormationProducer(w, h, cap_radii=cap_radii, cap_mass=1)
X, R, M, team_mapping = formation_producer.setup_match_formation("formation1", "formation1")


In [68]:
ur.render_field_snapshot(X, R, margin=margin)

### Move one cap with Velocity

In [78]:
V = np.zeros_like(X)
cap_move_index = 1
V[cap_move_index] = [15, -1]

In [79]:
motion = Motion(
    R=R,
    M=M,
    min_velocity=0.1,
    boundaries=boundaries,
    goal_size=goal_size,
    goal_depth=goal_depth
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

System stopped at iteration 800


In [80]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=5)

# Cap Remover from Goal

In [71]:
from match.match_logic.main_match import Match
match = Match()
# Import classes
ur = utils.UtilsRender()
ur = utils.UtilsRender(window_size=match.boundaries, goal_depth=match.goal_depth, goal_size=match.goal_size)

In [72]:
X = match.initial_setup(left_formation="formation2", right_formation="formation2")
R = match.R

In [73]:
ur.render_field_snapshot(position=X, radius=R)

In [74]:
X = np.array([[ 132.41379,  381.17648],
       [ 132.41379,  698.82355],
       [ 132.41379,  520.17648  ],
       [ -12.41379,  520.17648],
       [ 728.2759 ,  889.41174],
       [1787.5862 ,  381.17648],
       [1787.5862 ,  698.82355],
       [1522.7585 ,  540.     ],
       [1191.7241 ,  190.58824],
       [1191.7241 ,  889.41174],
       [ 960.     ,  540.     ]], dtype=np.float32)

In [31]:
ur.render_field_snapshot(position=X, radius=R)

In [75]:
g = match.goal_size
w,h = match.boundaries
TRP = (w, (h - g) / 2)
TLP = (0, (h - g) / 2)
BLP = (0, (h + g) / 2)
xfield_left = TLP[0]
xfield_right = TRP[0]
ynet_top_lateral = TLP[1]
ynet_bottom_lateral = BLP[1]
y_mid_field = (ynet_top_lateral + ynet_bottom_lateral) / 2

In [76]:
Xf = X.copy()

In [77]:
# Check which of the caps x coordinate is less than xfield_left
mask_caps_inside_left_goal = X[:, 0] < xfield_left
mask_caps_inside_right_goal = X[:, 0] > xfield_right

In [109]:
def remove_cap_inside_goal(x,y,r, kx, ky):
    """
    Check the angle of the y_cap_inside_goal. We should map:
    If the y_cap_inside_goal = ynet_top_lateral, angle = 45
    If the y_cap_inside_goal = ynet_bottom_lateral, angle = -45
    If the y_cap_inside_goal = y_mid_field, angle = 0
    Now we will displace the cap outside the goal (for left net we will add, for right net we will subtract on the x component)
    and we will displace the cap on the x axis by K * radii on the x axis and tan(angle) * radii on the y axis
    """
    angle = 70 * (y - y_mid_field) / (ynet_top_lateral - y_mid_field)
    return x + kx * r, y + ky * r * np.tan(np.radians(angle))

In [110]:
def select_kx_ky(cap_idx_in_goal, x_cap_inside_goal, y_cap_inside_goal, r):
    kx = 4
    ky = 4
    # Displace the cap outside the goal
    Xf[cap_idx_in_goal] = remove_cap_inside_goal(x_cap_inside_goal, y_cap_inside_goal, r, kx, ky)

    # Check if the updated position collides with any other cap
    check_collision_after_displacement = len(match.motion.cr.get_cap_collisions(Xf)) > 0

    #if len(match.motion.cr.get_cap_collisions(Xf)):
    # Create a for loop modifying  the kx argument until we find a valid position
    # In case we don't find a valid position, we will loop for the ky argument
    while check_collision_after_displacement:
        for kx in [3.5, 4, 4.5, 5, 5.5]:
            Xf[cap_idx_in_goal] = remove_cap_inside_goal(x_cap_inside_goal, y_cap_inside_goal, r, kx, ky)
            check_collision_after_displacement = len(match.motion.cr.get_cap_collisions(Xf)) > 0
            if not check_collision_after_displacement:
                break
        ky += 0.25
    return kx, ky


In [111]:
for cap_idx_in_goal in np.where(mask_caps_inside_left_goal)[0]:
    x_cap_inside_goal, y_cap_inside_goal = X[cap_idx_in_goal] 

    # Get the displacements factors allowed to avoid collisions
    kx, ky = select_kx_ky(cap_idx_in_goal, x_cap_inside_goal, y_cap_inside_goal, R[cap_idx_in_goal])
    Xf[cap_idx_in_goal] = remove_cap_inside_goal(x_cap_inside_goal, y_cap_inside_goal, R[cap_idx_in_goal], kx, ky)

In [113]:
# Given X and Xf, create an interpolation trajectory (simple, just linear) to create the motion of the caps moving from the X to Xf
# Use 100 timesteps in between
timesteps = 100
Xs = np.zeros((timesteps, X.shape[0], 2))
for t in range(timesteps):
    Xs[t] = X + (Xf - X) * t / timesteps


In [114]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=5)


In [67]:
ur.render_field_snapshot(position=X, radius=R)


In [112]:
ur.render_field_snapshot(position=Xf, radius=R)